# Test Seq Greedy Hueristic

TJ Kim
30 June 2020

Building new seq greedy heuristic solver class similar to ILP.

In [41]:
# Import Generic Classes
import numpy as np
import copy
import pickle

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/classes")

from Server import *
from User import *
from Link import *
from Job import *

from Migration_Plans import *

# Import Solver Classes
from Optim_PlanGenerator import *

In [57]:
# Migration Plan Code goes here
import time
import itertools

# Dijkstra's Algorithm
from Dijkstra_Graph import *

class SeqGreedy_PlanGenerator(PlanGenerator):
    """
    Generate migration plans with basic heuristic approach.
    """
    def __init__(self, users, servers, links, jobs, sim_params):
        
        # Store all relevant parameters within class
        super().__init__(users=users, servers=servers, links=links, jobs=jobs, sim_params=sim_params)
        
        # Components of subclass
        self.convert_node2st = None
        self.convert_st2node = None
        self.valid_links = None
        self.num_edges = None
        self.all_costs = None
        self.edge_weights_min = None
        
        # Build components above
        self.build_mig_graph()
        self.calc_all_costs()
        self.obtain_minimum_cost()
    
    def build_mig_graph(self):
        """
        Build the conversion table for all users that maps nodes in each user's migration graph 
        to a specific timestep and server (for job to be placed at)
        
        Also use this to make valid links table (matrix w 1 for valid, 0 for no edge)
        """
        
        self.convert_node2st = []
        self.convert_st2node = []
        self.valid_links = []
        self.num_edges = []
        
        for j in range(len(self.jobs)):
            
            # --------- Build Conversion Table ----------#
            conv_idx = 0
            active_time = self.jobs[j].active_time
            u_node2st = {} # Nested dictionary
            
            # Loop through each timestep to make node
            # Start node 
            u_node2st[conv_idx] = (-1,-1) # (s,t)
            conv_idx += 1
            
            # All Other Nodes
            for t in range(self.sim_params.time_steps):
                if active_time[t] > 0:
                    for s in range(len(self.servers)):
                        u_node2st[conv_idx] = (s,t)
                        conv_idx += 1
                else:
                    u_node2st[conv_idx] = (-1,t) # (s,t)
                    conv_idx += 1
            
            # End Node
            u_node2st[conv_idx] = (-1, self.sim_params.time_steps) #(s,t)
            
            u_st2node = dict([(value, key) for key, value in u_node2st.items()]) 
            self.convert_st2node += [u_st2node]
            self.convert_node2st += [u_node2st]
            
            
            # --------- Build Transition Matrix ----------#
            u_valid_links = np.zeros((len(u_node2st),len(u_node2st)))
            u_num_edges = np.zeros((len(u_node2st),len(u_node2st)))
            
            # Loop through every single node in the dictionary
            for key in u_st2node.keys():         
                server, active, time = key[0], key[0]  > -1, key[1]
                time_check = time >= 0 and time < self.sim_params.time_steps
                source_node = u_st2node[key]
                
                if time_check and active: # Source is active
                    max_edge_length = self.sim_params.max_edge_length
                    max_time = max(time+1, min(self.sim_params.time_steps, time + max_edge_length))
                    
                    for t2 in range(time+1,max_time+1):
                        if (t2 < self.sim_params.time_steps) and (self.jobs[j].active_time[t2] > 0):
                            for s in range(len(self.servers)):
                                if s != server:
                                    dest_node = u_st2node[(s,t2)]
                                    u_valid_links[source_node,dest_node] = 1
                                    u_num_edges[source_node,dest_node] = self.links.num_path[server,s]
                                elif s == server and t2 == time + 1:
                                    dest_node =u_st2node[(s,t2)]
                                    u_valid_links[source_node,dest_node] = 1
                                    u_num_edges[source_node,dest_node] = 1
                        elif (t2 == self.sim_params.time_steps) or self.jobs[j].active_time[t2] == 0:
                            if t2 == time + 1:
                                s = -1
                                dest_node = u_st2node[(s,t2)]
                                u_valid_links[source_node,dest_node] = 1
                                u_num_edges[source_node,dest_node] = 1
                            break # Break the loop as all other nodes don't matter
                        
                    
                # Source is inactive
                elif (time_check and not active) or (time == -1): 
                    time2 = time + 1
                    if time2 >= self.sim_params.time_steps:
                        active_2 = 0
                    else:
                        active_2 = self.jobs[j].active_time[time2]
                    
                    if active_2:
                        for s in range(len(self.servers)):
                            dest_node = u_st2node[(s,time2)]
                            u_valid_links[source_node,dest_node] = 1
                            u_num_edges[source_node,dest_node] = 1
                    else: # Both source and destination are inactive
                        dest_node = u_st2node[(-1,time2)]
                        u_valid_links[source_node,dest_node] = 1
                        u_num_edges[source_node,dest_node] = 1

            self.valid_links += [u_valid_links]
            self.num_edges += [u_num_edges]
            
    def calc_all_costs(self):
        """
        For every edge and path variation, calculate the cost and store in
        dictionary
        """
        
        self.all_costs = {}
        
        for j in range(len(self.jobs)):
            one_coor = zip(*np.where(self.valid_links[j] == 1))
            job_placement_rsrc = jobs[j].placement_rsrc
            job_migration_rsrc = jobs[j].migration_rsrc
            
            # Make Dictionary for job j key = [node1, node2, linkid], val = cost
            dict_n1n2 = {} 
            
            for (node1, node2) in one_coor:
                (server1, time1) = self.convert_node2st[j][node1]
                (server2, time2) = self.convert_node2st[j][node2]
                num_path = self.num_edges[j][(node1,node2)]
                
                (active1, active2) = (server1 > -1, server2 > -1)
                time_diff = time2-time1
                
                # Case 1 - Active to Active
                if active1 and active2:
                    # Placement and Migration Cost
                    placement_cost_s1 = np.dot(job_placement_rsrc,servers[server1].svr_rsrc_cost)*time_diff
                    if server1 != server2:
                        placement_cost_s2 = np.dot(job_placement_rsrc,servers[server2].svr_rsrc_cost)*time_diff
                        migration_cost = []
                        for n in range(int(num_path)):
                            num_path_links = self.links.get_subpath(server1,server2,n)
                            path_mig_cost = self.links.cost_links * num_path_links
                            migration_cost += [np.sum(np.sum(path_mig_cost,axis=1),axis=0)]
                        
                    else:
                        placement_cost_s2 = 0
                        migration_cost = [0]
                    
                    # Service BW Cost - Expectation of user loc
                    service_bw_cost = 0
                    curr_latency = 0
                    latency_cost = 0
                    
                    if time_diff > 1:
                        latency_list = []
                        for t in range(time1,time2):
                            temp_sbw, temp_cL = self.service_latency_cost(j,server1,t)
                            latency_list += [temp_cL]
                            service_bw_cost += temp_sbw
                        
                        temp_sbw, temp_cL = self.service_latency_cost(j,server2,time2)
                        service_bw_cost += temp_sbw
                        latency_list += [temp_cL]
                        
                        for t in range(time_diff):
                            leftover_latency = latency_list[t] - self.jobs[j].latency_req
                            if leftover_latency > 0:
                                latency_cost += self.jobs[j].latency_penalty * leftover_latency

                    elif time_diff == 1:
                        service_bw_cost, curr_latency = self.service_latency_cost(j,server2,time2)
                        
                        leftover_latency = curr_latency - self.jobs[j].latency_req
                        if leftover_latency > 0:
                            latency_cost = self.jobs[j].latency_penalty * leftover_latency
                    
                    # Record cost
                    cost_etc = placement_cost + placement_cost_s2 + service_bw_cost + latency_cost
                    for n in range(int(num_path)):
                        dict_n1n2[(node1,node2,n)] = cost_etc + migration_cost[n]
                
                # Case 2 - Inactive to Active
                elif (not active1) and active2:
                    placement_cost = np.dot(job_placement_rsrc,servers[server2].svr_rsrc_cost) 
                    
                    # Service BW Cost - Expectation of user loc
                    service_bw_cost, curr_latency = self.service_latency_cost(j,server2,time2)
                    
                    leftover_latency = curr_latency - self.jobs[j].latency_req
                    latency_cost = 0
                    if leftover_latency > 0:
                        latency_cost = self.jobs[j].latency_penalty * leftover_latency
                    
                    cost = placement_cost + service_bw_cost + latency_cost
                    dict_n1n2[(node1,node2,0)] = cost
                
                # Case 3 - Inactive to Inactive
                elif (not active1) and (not active2):
                    cost = 1
                    dict_n1n2[(node1,node2,0)] = cost
                
                # Case 4 - Active to Inactive
                elif active1 and (not active2):
                    cost = 1
                    dict_n1n2[(node1,node2,0)] = cost
          
                self.all_costs[j] = dict_n1n2
        
    # Subcost helper for latency and service bw
    def service_latency_cost(self,j,server,t):

        service_bw_cost = 0
        curr_latency = 0
        for s_var in range(len(self.servers)):
            if s_var != server:
                avg_link = self.links.get_avgpath(s_var,server)

                usr_job_flag = self.users[j].server_prob[s_var,t]
                expected_link_cost = np.multiply(self.links.cost_links, 
                                                 avg_link)
                total_link_cost = np.sum(np.sum(expected_link_cost,axis=1),axis=0)
                service_bw_cost += self.jobs[j].thruput_req * usr_job_flag * total_link_cost

                for s3, s4 in itertools.product(range(len(self.servers)),range(len(self.servers))):
                    delay = self.links.switch_delay + self.links.dist_delay * self.links.get_distance(s3,s4)
                    curr_latency += avg_link[s3,s4] * delay

        return service_bw_cost, curr_latency
            

    def obtain_minimum_cost(self):
        """
        For all users, calculate the minimum cost for every single edge for cost matrix
        eg. if between 2 servers there are multiple links, take the value with lower value
        """
        
        self.edge_weights_min = {}
        
        for j in range(len(jobs)):
            self.obtain_minimum_cost_j(j)
        
    def obtain_minimum_cost_j(self,j):
        """
        For a specific user u
        """
        output = np.zeros(self.valid_links[j].shape)
        cost_dict = self.all_costs[j]
        
        one_coor = zip(*np.where(self.valid_links[j] == 1))
        
        for node1,node2 in one_coor:
            num_path = self.num_edges[j][(node1,node2)]
            edges_list = []
            
            # Obtain all costs in an array
            for n in range(int(num_path)):
                edges_list += [cost_dict[(node1,node2,n)]]
                
            m = min(i for i in edges_list if i > 0)
        
            output[node1,node2] = m
        
        self.edge_weights_min[j] = output
    
    def dijkstra_j(self,j):
        """
        Find shortest point between start and end node 
        """
        
        edge_weights = self.edge_weights_min[j]
        d_graph = Dijkstra_Graph()
        
        one_coor = zip(*np.where(self.valid_links[j] == 1))
        
        start_node = self.convert_st2node[j][(-1,-1)]
        end_node = self.convert_st2node[j][(-1,self.sim_params.time_steps)]
        
        for node1,node2 in one_coor:
            weight = edge_weights[node1,node2]
            d_graph.add_edge(node1,node2,weight)
        
        shortest_path = dijsktra(d_graph,start_node,end_node)
        
        return shortest_path
    
    

### Make Simulation Environment and Create Job, Server, Users

In [58]:
"""
Make Simulation Parameters
"""
sim_param = Sim_Params(time_steps=3, x_length = 5, y_length = 5, max_edge_length=10)
boundaries = np.array([[0,sim_param.x_length],[0,sim_param.y_length]])


"""
Make Job Profiles
"""
# REsources used are CPU (no. cores) storage (GB), and RAM (GB)
# througput is in mb/s
# Latency is in ms

job_profile1 = Job_Profile(job_name = "VR",
                           latency_req_range=[25, 40], 
                           thruput_req_range=[50/1000, 200/1000], 
                           length_range=[sim_param.time_steps,sim_param.time_steps],  
                           placement_rsrc_range = np.array([[2,3],[8,16],[2,5]]),
                           migration_amt_range = [5, 10],
                           latency_penalty_range = [0.05, 0.1]) 

job_profile2 = Job_Profile(job_name = "Assistant",
                           latency_req_range=[100, 200],
                           thruput_req_range=[5/1000, 20/1000],
                           length_range=[sim_param.time_steps,sim_param.time_steps],
                           placement_rsrc_range = np.array([[1,1],[0.5,1],[0.5,1]]),
                           migration_amt_range = [0.5, 1],
                           latency_penalty_range = [0.01, 0.05])

job_profile3 = Job_Profile(job_name = "AR",
                           latency_req_range=[1000, 1000], 
                           thruput_req_range=[1,1],
                           length_range=[sim_param.time_steps,sim_param.time_steps],
                           placement_rsrc_range = np.array([[1,1]]),
                           migration_amt_range = [1, 1],
                           latency_penalty_range = [0.03, 0.08])

job_profiles = [job_profile1, job_profile2, job_profile3]


"""
Make Servers
"""

# Server Settings
num_server_l1 = 0
num_server_l2 = 2
num_server_l3 = 0

num_resource = 1
weak_range = np.array([[4,8]])
strong_range = np.array([[50,100]])

rsrc_cost = np.array([1])
rsrc_cost_scale_lv1 = 2
rsrc_cost_scale_lv2 = 1
rsrc_cost_scale_lv3 = 1

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []
idx_counter = 0

for i in range(num_server_l1):
    servers_l1.append(Server(boundaries,level=1,rand_locs=True,locs=None))
    servers_l1[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l1[-1].assign_id(idx_counter)
    servers_l1[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv1)
    idx_counter += 1
    
for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    servers_l2[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l2[-1].assign_id(idx_counter)
    servers_l2[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv2)
    idx_counter += 1
    
for i in range(num_server_l3):
    servers_l3.append(Server(boundaries,level=3,rand_locs=False,locs=np.array([200,200])))
    servers_l3[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l3[-1].assign_id(idx_counter)
    servers_l3[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv3)
    idx_counter += 1
    
servers = servers_l1 + servers_l2 + servers_l3


"""
Make Links
"""

# Link Settings
num_link = [0,1,2,3]
prob_link = [0.5,0.2,0.2,0.1]
lv_minmax = np.array(([[500,1000],[10000,20000],[30000,50000]]))
lv1_transmission = 1
link_costs = np.array([1, 1, 1])
latency_settings = [10, 1] #[ms per switch, ms per mile]

links = Link(servers, num_link, prob_link, lv_minmax, link_costs, latency_settings,lv1_transmission)


"""
Make Users
"""

# User Settings
num_user_m0 = 0 # Pedestrian
num_user_m1 = 0 # Public Transport
num_user_m2 = 1 # Vehicle

max_speed = 2.5
lamdas = [1/0.25,1/0.83,1/1.67] # 3 mph, 10 mph, 20 mph
num_path = 10

# Generate Server
users_m0 = []
users_m1 = []
users_m2 = []
idx_counter = 0

for i in range(num_user_m0):
    users_m0 += [User(boundaries, sim_param.time_steps, 0, lamdas, max_speed, num_path)]
    users_m0[-1].generate_MC(servers)
    users_m0[-1].assign_id(idx_counter)
    idx_counter += 1
    
for i in range(num_user_m1):
    users_m1 += [User(boundaries, sim_param.time_steps, 1, lamdas, max_speed, 1)]
    users_m1[-1].generate_MC(servers)
    users_m1[-1].assign_id(idx_counter)
    idx_counter += 1

for i in range(num_user_m2):
    users_m2 += [User(boundaries, sim_param.time_steps, 2, lamdas, max_speed, num_path)]
    users_m2[-1].generate_MC(servers)
    users_m2[-1].assign_id(idx_counter)
    idx_counter += 1

users = users_m0 + users_m1 + users_m2
    
    
"""
Make Jobs
- "I'm just going to do it"
"""

# Job settings
job_type0 = 0 # VR
job_type1 = 0 # Assistant
job_type2 = 1 # AR

jobs0 = []
jobs1 = []
jobs2 = []
idx_counter = 0

total_job_count = job_type0+job_type1+job_type2
draw_job_id = np.random.choice(total_job_count, total_job_count, replace=False)

for i in range(job_type0):
    jobs0 += [Job(job_type = 0,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles = job_profiles)]
    idx_counter += 1
    
for i in range(job_type1):
    jobs1 += [Job(job_type = 1,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles = job_profiles)]
    idx_counter += 1
    
for i in range(job_type2):
    jobs2 += [Job(job_type = 2,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles=job_profiles)]
    idx_counter += 1
    
jobs = jobs0 + jobs1 + jobs2

In [59]:
jobs[0].active_time = np.array([1,1,1])

sg_plan1 = SeqGreedy_PlanGenerator(users, servers, links, jobs, sim_param)

In [60]:
sg_plan1.edge_weights_min[0]

array([[0. , 1. , 2. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 1.2, 3.8, 0. , 5. , 0. ],
       [0. , 0. , 0. , 3.2, 1.8, 6. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1.2, 3.8, 0. ],
       [0. , 0. , 0. , 0. , 0. , 3.2, 1.8, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [67]:
print(sg_plan1.dijkstra_j(0))

[0, 1, 3, 5, 7]


### Check if edge generation works properly

5 timestep scenario with 3 active in the middle, 3 servers with 2 numpath so that we can check both long term migrations and multiple paths between two servers.

In [6]:
sg_plan1.calc_all_costs()

In [9]:
sg_plan1.num_edges[0]

array([[0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 1., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [65]:
sg_plan1.all_costs[0]

{(0, 1, 0): 1.0,
 (0, 2, 0): 2.0,
 (1, 3, 0): 1.2,
 (1, 4, 0): 3.8,
 (1, 6, 0): 5.0,
 (2, 3, 0): 3.2,
 (2, 4, 0): 1.8,
 (2, 5, 0): 6.0,
 (3, 5, 0): 1.2,
 (3, 6, 0): 3.8,
 (4, 5, 0): 3.2,
 (4, 6, 0): 1.8,
 (5, 7, 0): 1,
 (6, 7, 0): 1}

In [66]:
# (s,t)
sg_plan1.convert_st2node[0]

{(-1, -1): 0,
 (0, 0): 1,
 (1, 0): 2,
 (0, 1): 3,
 (1, 1): 4,
 (0, 2): 5,
 (1, 2): 6,
 (-1, 3): 7}

In [70]:
users[0].server_prob

array([[1. , 0.8, 0.8],
       [0. , 0.2, 0.2]])